# VERİNİN TEMİZLENMESİ SÜRECİ 

In [105]:
import pandas as pd

In [106]:
df = pd.read_parquet("steam.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83560 entries, 0 to 83559
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       83560 non-null  int64  
 1   Name                        83554 non-null  object 
 2   Release date                83560 non-null  object 
 3   Estimated owners            83560 non-null  object 
 4   Peak CCU                    83560 non-null  int64  
 5   Required age                83560 non-null  int64  
 6   Price                       83560 non-null  float64
 7   DLC count                   83560 non-null  int64  
 8   About the game              80123 non-null  object 
 9   Supported languages         83560 non-null  object 
 10  Full audio languages        83560 non-null  object 
 11  Reviews                     9716 non-null   object 
 12  Header image                83560 non-null  object 
 13  Website                     390

In [107]:
# Eksik değer kontrolü
print("Eksik değerler: \n")
df.isna().sum().sort_values(ascending=False).head(15)

Eksik değerler: 



Score rank        83516
Metacritic url    79650
Reviews           73844
Notes             70845
Website           44506
Support url       42544
Tags              19986
Support email     13319
Movies             6300
Categories         4456
Publishers         3705
Developers         3457
About the game     3437
Genres             3425
Screenshots        1926
dtype: int64

## BİLGİ KAZANCI KONTROLÜ
Bilgi kazancı çok az olan sütunları çıkaralım.

In [108]:

zero_age_count = (df['Required age'] == 0).sum()
print(f"Required age'deki 0 olan değerlerin sayısı: {zero_age_count}")
zero_metacriticscore_count = (df['Metacritic score'] == 0).sum()
print(f"Metacritic score'daki 0 olan değerlerin sayısı: {zero_metacriticscore_count}")
zero_user_score_count = (df['User score'] == 0).sum()
print(f"User score'daki 0 olan değerlerin sayısı: {zero_user_score_count}")
zero_peak_ccu_count = (df['Peak CCU'] == 0).sum()
print(f"Peak ccu'daki 0 olan değerlerin sayısı: {zero_peak_ccu_count}")
zero_dlc_count = (df['DLC count'] == 0).sum()
print(f"Dlc count'daki 0 olan değerlerin sayısı: {zero_dlc_count}")
zero_rec_count = (df['Recommendations'] == 0).sum()
print(f"Recommendations'daki 0 olan değerlerin sayısı: {zero_rec_count}")
# Hem Positive hem Negative 0 olan satırları bul
both_zero_mask = (df['Positive'] == 0) & (df['Negative'] == 0 & (df['Average playtime two weeks'] == 0) & (df['Average playtime forever'] == 0))
both_zero_count = both_zero_mask.sum()
print(f"Hem Positive hem Negative hem de Average palay time'ları 0 olan oyun sayısı: {both_zero_count:,} ({both_zero_count/len(df)*100:.2f}%)")
zero_ach_count = (df['Average playtime forever'] == 0).sum()
print(f"Average playtime forever'daki 0 olan değerlerin sayısı: {zero_ach_count}")
zero_ach_count = (df['Average playtime two weeks'] == 0).sum()
print(f"Average playtime two weeks'daki 0 olan değerlerin sayısı: {zero_ach_count}")

# Bu satırları veri çerçevesinden çıkar
df = df[~both_zero_mask].copy()
print(f"\nTemizlenmiş dataset boyutu: {df.shape}")



Required age'deki 0 olan değerlerin sayısı: 81930
Metacritic score'daki 0 olan değerlerin sayısı: 79650
User score'daki 0 olan değerlerin sayısı: 83516
Peak ccu'daki 0 olan değerlerin sayısı: 61189
Dlc count'daki 0 olan değerlerin sayısı: 71798
Recommendations'daki 0 olan değerlerin sayısı: 69837
Hem Positive hem Negative hem de Average palay time'ları 0 olan oyun sayısı: 20,053 (24.00%)
Average playtime forever'daki 0 olan değerlerin sayısı: 68656
Average playtime two weeks'daki 0 olan değerlerin sayısı: 81512

Temizlenmiş dataset boyutu: (63507, 39)


In [109]:
# Çok fazla eksik değer olanları ve popülerlik 
# tahminindeki bilgi kazancı az olan öznitelikleri tablodan çıkardık.
df = df.drop(columns=["Score rank", "Metacritic url", "Reviews", "Notes", 
                      "Website", "Support url", "Name", "Support email", 
                      "Header image", "Movies", "Screenshots", 
                      "About the game", "Full audio languages", "Windows", 
                      "Linux", "Mac", "Metacritic score", "User score", "Required age",
                      "Peak CCU", "DLC count", "Supported languages", "Recommendations",
                      ])

In [110]:
# Kategorik sütunlardaki boş veirleri uygun olarak dolduralım
df["Tags"] = df["Tags"].fillna("Unknown")
df["Categories"] = df["Categories"].fillna("Unknown") 
df["Publishers"] = df["Publishers"].fillna("Unknown Publisher")
df["Developers"] = df["Developers"].fillna("Unknown Developer")
df["Genres"] = df["Genres"].fillna("Unknown")

def clean_developer_and_publishers(dev):
    dev = dev.strip().lower()  # Baş/son boşlukları sil ve küçük harfe çevir
    dev = dev.replace('.', '')  # Noktaları çıkar

    # Şirket tipi keyword'lerini temizle (öncesinde varsa virgül de)
    for keyword in ['llc', 'ltd', 'inc']:
        # Eğer keyword geçiyorsa
        if keyword in dev:
            # Öncesinde virgül ve boşluk varsa çıkar
            dev = dev.replace(f', {keyword}', '')
            dev = dev.replace(f' {keyword}', '')
            dev = dev.replace(f',{keyword}', '')
            dev = dev.replace(keyword, '')  # Kalan varsa onu da çıkar

    return dev.strip()

# Uygula
df['Developers'] = df['Developers'].apply(clean_developer_and_publishers)
df['Publishers'] = df['Publishers'].apply(clean_developer_and_publishers)

In [111]:
df.isna().sum().sort_values(ascending=False).head(3)

AppID               0
Release date        0
Estimated owners    0
dtype: int64

In [112]:
df.to_parquet("cleaned_data.parquet", index=False)